# PROBLEM 1

In [44]:
from random import uniform
from math import sqrt
from time import time
import numpy

In [2]:
number_of_darts = 200000

In [10]:
def throw_dart():
    x,y = uniform(0,1), uniform(0,1)
    if sqrt((x-0.5)**2+(y-0.5)**2) <= 0.5:
        return 1
    return 0

In [11]:
def count_darts_in_circle_serial(n):
    number_of_darts_in_circle = 0
    for n in range(n):
        number_of_darts_in_circle += throw_dart()
    return number_of_darts_in_circle

In [12]:
def runtime(method, func, n):
    start_time = time()
    count = func(n)
    end_time = time()
    execution_time = end_time - start_time
    pi_approx = 4*count/n
    print(method, '\n')
    print('Pi Approximation:', pi_approx)
    print('Number of Darts:', n)
    print('Execution Time (s):',execution_time)
    print('Darts Thrown per Second:', number_of_darts/execution_time)

In [13]:
runtime('SIMPLE SERIAL', count_darts_in_circle_serial, number_of_darts)

SIMPLE SERIAL 

Pi Approximation: 3.14718
Number of Darts: 200000
Execution Time (s): 0.37608885765075684
Darts Thrown per Second: 531789.2193065814


In [14]:
import multiprocessing

np = multiprocessing.cpu_count()
print('You have {0:1d} CPUs'.format(np))

You have 4 CPUs


In [32]:
def count_darts_in_circle_multi(n):
    part_count = [int(n/np) for i in range(np)]
    p = multiprocessing.Pool(np)
    count = p.map(count_darts_in_circle_serial, part_count)
    number_of_darts_in_circle = sum(count)
    return number_of_darts_in_circle

In [33]:
runtime('MULTIPROCESSING', count_darts_in_circle_multi, number_of_darts)

MULTIPROCESSING 

Pi Approximation: 3.13478
Number of Darts: 200000
Execution Time (s): 0.22660279273986816
Darts Thrown per Second: 882601.6554420526


In [36]:
import dask as da

def count_darts_in_circle_da(n):
    count = [da.delayed(throw_dart)() for i in range(n)]
    number_of_darts_in_circle = sum(da.compute(*count))
    return number_of_darts_in_circle

In [37]:
runtime('DASK', count_darts_in_circle_da, number_of_darts)

DASK 

Pi Approximation: 3.14628
Number of Darts: 200000
Execution Time (s): 54.93285393714905
Darts Thrown per Second: 3640.808471899681


Tried making this work, will come back latter if I have time.   
```python
import ipyparallel as ipp  
rc = ipp.Client()  
ipcluster nbextension enable  
```

# PROBLEM 2

In [46]:
def running_trial(func,ndarts):
    maxdarts = numpy.max(ndarts)
    start = time()
    count = func(ndarts[0])
    exec_time = time()-start
    times = [exec_time]
    for i,n in enumerate(ndarts[1:]):
        count = func(n-ndarts[i])
        exec_time = time()-start
        if n in ndarts:
            times.append(exec_time)
    return times

def many_trials(func,ndarts,ntrials=20):
    trials = numpy.array([running_trial(func,ndarts) for _ in range(ntrials)])
    return trials

def mean_time(trials):
    return numpy.mean(trials,axis=0)

def ci_time(trials):
    low = stats.scoreatpercentile(trials,5,axis=0)
    high = stats.scoreatpercentile(trials,95,axis=0)
    return low,high

In [ ]:
ndarts = numpy.logspace(1, 8, 20, dtype=int)
serial_times = running_trial(count_darts_in_circle_serial, ndarts)
multi_times = running_trial(count_darts_in_circle_multi, ndarts)
da_times = running_trial(count_darts_in_circle_da, ndarts)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(8,6))
plt.loglog(ndarts,serial_times, label='serial')
plt.loglog(ndarts,multi_times, label='multiprocessing')
plt.loglog(ndarts,da_times, label='dask')
plt.legend(loc=2)
plt.ylabel('Time [s]')
plt.xlabel('Darts Thrown')
plt.savefig('plots/dart.png')
plt.show()

# COMPUTER INFO
MacBook Pro  
2.4 GHz Intel Core i5  
4 "cores"  